### Classification and Predicition - Full_Phecode_ATC_Top100_180days

In [1]:
import re
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import random
from IPython.display import display
from IPython.display import Image
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, accuracy_score,confusion_matrix,precision_score,recall_score,f1_score
from sklearn.metrics import classification_report

from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt

#import os
#from dotenv import load_dotenv
#load_dotenv()


######### Split Medication and Diagnosis In Unique Pieces ############
#                                                                    #
#                                                                    #
#                                                                    #
######################################################################

class Full_Phecode_ATC_ClassifyReadmissionWithMedicationDiagnosis:
    def __init__(self, file1):
        self.file1 = file1
        self.merged_df = None
        self.selected_column_merged_df = None
        self.without_diag_medic_selected_column_merged_df = None
    
    def load_data(self):
        Full_Phecode_ATC = pd.read_csv(self.file1)       
        self.merged_df = Full_Phecode_ATC[['episode_id', 'num_diagnoses', 'num_medications', 'pasient', 'age',
       'remaining_time_countdown', 'var_no_dates_permonth', 'gender',
       'episode_order', 'islast', 'closingcode', 'aftercode',
       'episode_start_date', 'episode_end_date', 'tillnextepisode',
       'Length_of_Episode', 'Cat_LOE', 'TNE_BO_180', 'TNE_NO_180',
       'TNE_BO_365', 'TNE_NO_365', 'TNE_BO_730', 'TNE_NO_730', 'TNE_BO_1095',
       'TNE_NO_1095', 'Cat_LOE_desc', 'Count_visit', 'Cat_CV', 'Therapy_ratio',
       'Examination_ratio', 'Advisory_ratio', 'TreatmentPlanning_ratio',
       'Outpatient_ratio', 'Inpatient_ratio', 'Inpatient_day_ratio',
       'Inpatient_daynight_ratio', 'Care_intensity', 'age_group',
       'closingcode_0', 'closingcode_1', 'closingcode_2', 'closingcode_3',
       'closingcode_4', 'closingcode_5', 'closingcode_6', 'closingcode_9',
       'aftercode_1', 'aftercode_2', 'aftercode_3', 'aftercode_4',
       'aftercode_5', 'gender_0', 'F', 'M', 'MiddleChildhood', 'Preschooler',
       'Teenager', 'actual_diag_Phe', 'actual_med_Full_ATC']]
        self.merged_df = self.merged_df.copy(deep=True)
        return self.merged_df


     ##### Split Diagnose and Medication, and save new dataframe  
    def split_diagnosis_medication(self):
        newmerged_df = Full_Phecode_ATC_ClassifyReadmissionWithMedicationDiagnosis_Obj.load_data()
        columns = ['actual_diag_Phe', 'actual_med_Full_ATC']
        lists_dict = {}
        diagnoses_list = []
        actual_med_list = [] 
        for col in columns:
            newmerged_df[col] = newmerged_df[col].apply(lambda d:[] if pd.isnull(d) else d)
            newmerged_df[col] = newmerged_df[col].str.replace("[", "")
            newmerged_df[col] = newmerged_df[col].str.replace("]", "")
            newmerged_df[col] = newmerged_df[col].str.replace("'", "")
            newmerged_df[col] = newmerged_df[col].str.replace(" ", "")
            newmerged_df[col] = newmerged_df[col].str.split(',')
            newmerged_df = newmerged_df.explode(col)
            
            # Show only top 100 columns names
            temp_store_100 = newmerged_df[col].value_counts().head(100)
            names_only = temp_store_100.index.tolist()
            lists_dict[col] = names_only
            #print(names_only)

            if col =='actual_diag_Phe':
                diagnoses_list = lists_dict[col]
            else:
                actual_med_list = lists_dict['actual_med_Full_ATC']

            original_columns = newmerged_df.columns.tolist()
            dummies = newmerged_df[col].str.get_dummies(sep=',')
            newmerged_df = pd.concat([newmerged_df, dummies], axis=1)
            newmerged_df = newmerged_df.drop(col, axis=1)

        newmerged_df.to_csv('/mnt/work/workbench/dipendrp/new-data/Unique_Full_Phecode_ATC.csv',index=False)
        concatenated_df = pd.concat([newmerged_df.iloc[:, :57], newmerged_df[diagnoses_list], newmerged_df[actual_med_list]], axis=1)
        concatenated_df.to_csv('/mnt/work/workbench/dipendrp/new-data/Top100_Full_Phecode_ATC.csv',index=False)  
       
    
class ClassifyReadmissionWithMedicationDiagnosis:
    def __init__(self, file2):
        self.file2 = file2
        self.merged_df = None
        self.selected_column_merged_df = None
    
    def load_data(self):
        self.merged_df = pd.read_csv(self.file2)
        columns_to_remove = ['pasient', 'age','episode_id','episode_order', 'gender', 'age_group', 'islast', 'closingcode', 'aftercode', 'Inpatient_ratio', 'Inpatient_day_ratio','episode_start_date', 'episode_end_date', 'tillnextepisode', 'Cat_LOE','Cat_LOE_desc', 'Cat_CV','TNE_BO_365', 'TNE_NO_365', 'TNE_BO_730', 'TNE_NO_730', 'TNE_BO_1095','TNE_NO_1095', 'nan']
        self.merged_df = self.merged_df.drop(columns=columns_to_remove)

        # fill nan values in independent column
        self.merged_df['num_diagnoses'].fillna(0, inplace=True)
        self.merged_df['num_medications'].fillna(0, inplace=True)
        self.merged_df['Outpatient_ratio'].fillna(0, inplace=True)
        self.merged_df['Inpatient_daynight_ratio'].fillna(0, inplace=True)  
        self.merged_df['Therapy_ratio'].fillna(0, inplace=True)
        self.merged_df['Examination_ratio'].fillna(0, inplace=True)
        self.merged_df['Advisory_ratio'].fillna(0, inplace=True)
        self.merged_df['TreatmentPlanning_ratio'].fillna(0, inplace=True)

        # define dependent variables
        self.dependent_variable_TNE_BO_180 = self.merged_df[['TNE_BO_180']]
        self.independent_variable = self.merged_df.drop(columns=['TNE_BO_180', 'TNE_NO_180'])

        columns_with_nan = self.independent_variable.columns[self.independent_variable.isnull().any()].tolist()

        return self.merged_df, self.dependent_variable_TNE_BO_180, self.independent_variable 
    
    # To train and view the reasult of binary and multiclass classifier        
    def train_classifier_with_medication_diagnosis(self):
        dependent_variables = [self.dependent_variable_TNE_BO_180]
        
        # to define the weight of output class, to resolve imbalanced output/dataset
        dependent_variable_names = {'self.dependent_variable_TNE_BO_180': 'TNE_BO_180'}
        class_weights = {'TNE_BO_180': {0:10, 1:270}}
              
        #for dependent_variable in dependent_variables:
        for variable_name, dependent_variable in zip(dependent_variable_names.values(), dependent_variables):
            logistic_prediction_model = LogisticRegression(class_weight=class_weights[variable_name])
            X_train, X_test, y_train, y_test = train_test_split(self.independent_variable, dependent_variable, train_size=0.7)
            logistic_prediction_model.fit(X_train,y_train)
            y_pred = logistic_prediction_model.predict(X_test)
            
            # Checks if the output category for classification is binary or multiclass
            category_count = int(y_train.nunique())
            print(f'\n****** Evaluation result {variable_name} as dependent variable ******')
            # For binary class classification 
            if category_count == 2:
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='binary')
                recall = recall_score(y_test, y_pred, average='binary')
                f1 = f1_score(y_test, y_pred, average='binary')
                
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1score: {f1}")
                print("Confused matrix:")
                conf_matrix = confusion_matrix(y_test, y_pred)
                tn, fp, fn, tp = conf_matrix.ravel()
                print(conf_matrix)
                print(f"True Negative (TN): {tn}")
                print(f"False Positive (FP): {fp}")
                print(f"False Negative (FN): {fn}")
                print(f"True Positive (TP): {tp}")
                print(classification_report(y_test, y_pred))
            
            # For binary class classification 
            else:
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred, average='weighted')
                recall = recall_score(y_test, y_pred, average='weighted')
                f1 = f1_score(y_test, y_pred, average='weighted')
                
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1score: {f1}")
                conf_matrix = confusion_matrix(y_test, y_pred)
                print("Confused matrix:")
                print(conf_matrix)
                print(classification_report(y_test, y_pred))
                
    def prediction_with_medication_diagnosis(self):
        # Select all corresponding columns with values in TNE_NO_180
        self.merged_df = self.merged_df.dropna(subset=['TNE_NO_180'])
        
        # define dependent variables
        self.dependent_variable_TNE_NO_180 = self.merged_df[['TNE_NO_180']]
        # define independent variables
        self.independent_variable = self.merged_df.drop(columns=['TNE_BO_180', 'TNE_NO_180'])

        dependent_variables = [self.dependent_variable_TNE_NO_180]
        dependent_variable_names = {'self.dependent_variable_TNE_NO_180':'TNE_NO_180'}
        #for dependent_variable in dependent_variables:
        for variable_name, dependent_variable in zip(dependent_variable_names.values(), dependent_variables):
            linear_prediction_model = LinearRegression()
            X_train, X_temp, y_train, y_temp = train_test_split(self.independent_variable, dependent_variable, train_size=0.7)
            X_test,X_eval,y_test,y_eval = train_test_split(X_temp,y_temp,test_size=0.33)
            linear_prediction_model.fit(X_train,y_train)
            y_pred = linear_prediction_model.predict(X_test)
            print(f'\n****** Prediction Model Evaluation result  {variable_name} as dependent variable ******')
            y_pred = linear_prediction_model.predict(X_test)
            r2 = r2_score(y_test, y_pred)
            mse = mean_squared_error(y_test, y_pred)
            print('R-squared:', r2)
            print('Mean squared error:', mse)
                

Full_Phecode_ATC_ClassifyReadmissionWithMedicationDiagnosis_Obj = Full_Phecode_ATC_ClassifyReadmissionWithMedicationDiagnosis('/mnt/work/workbench/dipendrp/new-data/Full_Phecode_ATC.csv')
Full_Phecode_ATC_ClassifyReadmissionWithMedicationDiagnosis_Obj.split_diagnosis_medication()

ClassifyReadmissionWithMedicationDiagnosis_Obj = ClassifyReadmissionWithMedicationDiagnosis('/mnt/work/workbench/dipendrp/new-data/Top100_Full_Phecode_ATC.csv')
merged_df, dependent_variable_TNE_BO_180, independent_variable  = ClassifyReadmissionWithMedicationDiagnosis_Obj.load_data()

ClassifyReadmissionWithMedicationDiagnosis_Obj.train_classifier_with_medication_diagnosis()
ClassifyReadmissionWithMedicationDiagnosis_Obj.prediction_with_medication_diagnosis()


****** Evaluation result TNE_BO_180 as dependent variable ******
Accuracy: 0.6493258426966292
Precision: 0.052129688493324854
Recall: 0.5412541254125413
F1score: 0.09510002899391126
Confused matrix:
[[5615 2982]
 [ 139  164]]
True Negative (TN): 5615
False Positive (FP): 2982
False Negative (FN): 139
True Positive (TP): 164
              precision    recall  f1-score   support

         0.0       0.98      0.65      0.78      8597
         1.0       0.05      0.54      0.10       303

    accuracy                           0.65      8900
   macro avg       0.51      0.60      0.44      8900
weighted avg       0.94      0.65      0.76      8900


****** Prediction Model Evaluation result  TNE_NO_180 as dependent variable ******
R-squared: -0.09434335855694775
Mean squared error: 2688.2374327099797


In [2]:
Full_Phecode_ATC = pd.read_csv('/mnt/work/workbench/dipendrp/new-data/Full_Phecode_ATC.csv') 
Full_Phecode_ATC[['actual_diag_Phe']]

,actual_diag_Phe
0,"['3001', '3009']"
1,NaN
2,['3895']
3,['29622']
4,NaN
...,...
17120,"['3131', '3132']"
17121,NaN
17122,"['3131', '3695', '312']"
17123,NaN
